In [5]:
import os,glob
import time
import copy
import math
import random
import csv
import json

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from file_io import load_image
from tree import get_region_name_list,check_region_u32list,get_region_rgb_list

def write_somaobj_fromtree(obj_dest, tree):
    f = open(obj_dest, 'w+')
    k = 0
    total_s = ''
    for leaf in tree:
        k = k+1
        i,t,x,y,z,r,p = leaf
        s = ['v ' ,str(x),' ',str(y),' ',str(z),'\n']
        f.writelines(s)
        total_s += f'p {k}\n'
    f.writelines(total_s)
    f.close()
    
def write_somaobj_fromxyzlist(obj_dest, xyzlist):
    f = open(obj_dest, 'w+')
    k = 0
    total_s = ''
    for xyz in xyzlist:
        k = k+1
        x,y,z = xyz
        s = ['v ' ,str(x),' ',str(y),' ',str(z),'\n']
        f.writelines(s)
        total_s += f'p {k}\n'
    f.writelines(total_s)
    f.close()

def get_n671_u32_region_list(regi_xyz):
    regi_xyz = np.array(regi_xyz).astype(int)
    
    n671_u32_mask = load_image("../assets/annotation_25.nrrd")
    mz,my,mx = n671_u32_mask.shape
    
    regi_xyz_new = regi_xyz.copy()
    regi_xyz_new[:,2][regi_xyz_new[:,2]>=mz] = mz-1
    regi_xyz_new[:,2][regi_xyz_new[:,2]<0] = 0
    regi_xyz_new[:,1][regi_xyz_new[:,1]>=my] = my-1
    regi_xyz_new[:,1][regi_xyz_new[:,1]<0] = 0
    regi_xyz_new[:,0][regi_xyz_new[:,0]>=mx] = mx-1
    regi_xyz_new[:,0][regi_xyz_new[:,0]<0] = 0
    index = (regi_xyz_new[:,2],regi_xyz_new[:,1],regi_xyz_new[:,0])
    region_n671_list = n671_u32_mask[index].tolist()
    
    region_n671_list = np.array(region_n671_list)
    region_n671_list[regi_xyz[:,2]>=mz] = 0
    region_n671_list[regi_xyz[:,2]<0] = 0
    region_n671_list[regi_xyz[:,1]>=my] = 0
    region_n671_list[regi_xyz[:,1]<0] = 0
    region_n671_list[regi_xyz[:,0]>=mx] = 0
    region_n671_list[regi_xyz[:,0]<0] = 0
        
    region_n671_list = region_n671_list.tolist()
    return region_n671_list
    
def get_xyz(file):
    if file.endswith('.swc'): delimiter = ' '
    elif file.endswith('.apo'): delimiter = ',' 
        
    with open(file,'r') as f:
        lastline = 'find#'
        ix,iy,iz = [-1,-1,-1]
        xyz = []
        for line in f.readlines():
            if lastline[0]=='#' and line[0]!='#':
                head = lastline.replace('#','').split(',')
                ix,iy,iz = [head.index(i) for i in ['x','y','z']]
            lastline = line
            
            if [ix,iy,iz] != [-1,-1,-1]:
                content = line.split(delimiter)
                xyz.append([float(content[i]) for i in [ix,iy,iz]])
    return xyz

def get_marker_num(file):
    out = os.popen(f'cat {file} | grep -v "#" | wc -l').read()
    num = int(out.strip())
    return num

In [6]:
soma_dir = '/PBshare/SEU-ALLEN/Projects/200k/somalist/22020608'
raw_soma_dir1 = soma_dir + '/raw_brain/refined/50k'
brains_raw_soma_list1 = [b.split('_')[0] for b in os.listdir(raw_soma_dir1)]
brains_raw_soma_list1.sort()
print(len(brains_raw_soma_list1),brains_raw_soma_list1)
raw_soma_dir2 = soma_dir + '/raw_brain/refined/150k'
brains_raw_soma_list2 = [b.split('_')[0] for b in os.listdir(raw_soma_dir2)]
brains_raw_soma_list2.sort()
print(len(brains_raw_soma_list2),brains_raw_soma_list2)
regi_soma_dir = soma_dir + '/registered'
brains_regi_soma_list = [b.split('_')[0] for b in os.listdir(regi_soma_dir)]
brains_regi_soma_list.sort()
print(len(brains_regi_soma_list),brains_regi_soma_list)

58 ['17051', '17052', '17109', '17300', '17301', '17302', '17304', '17539', '17541', '17542', '17543', '17544', '17545', '17781', '17782', '17783', '17785', '17786', '17787', '17788', '18047', '18049', '18052', '18053', '182712', '182726', '18452', '18453', '18454', '18455', '18457', '18458', '18459', '18461', '18462', '18463', '18464', '18465', '18466', '18467', '18468', '18469', '18470', '18471', '18472', '18860', '18861', '18862', '18864', '18865', '18866', '18867', '18868', '18869', '18872', '191804', '191808', '236174']
58 ['15257', '17298', '182721', '182722', '191797', '191798', '191799', '191813', '191817', '192333', '192334', '192337', '192341', '192343', '192344', '192346', '192348', '192349', '194060', '194061', '194062', '194063', '194064', '194069', '194072', '194073', '194075', '194076', '194077', '194078', '194080', '194081', '196463', '196464', '196465', '196466', '196468', '196469', '196470', '196471', '196472', '196474', '196476', '196477', '196478', '201582', '201584

In [7]:
soma_refined2regi = []
for brain in brains_regi_soma_list:
    if brain in brains_raw_soma_list1:
        brain_soma_file = f'{regi_soma_dir}/{brain}_total_stps.swc'
        raw_brain_soma_file = f'{raw_soma_dir1}/{brain}_total.apo'
    if brain in brains_raw_soma_list2:
        brain_soma_file = f'{regi_soma_dir}/{brain}_refined_stps.swc'
        raw_brain_soma_file = f'{raw_soma_dir2}/{brain}_refined.apo'
    raw_num = get_marker_num(raw_brain_soma_file)
    regi_num = get_marker_num(brain_soma_file)
    if raw_num!=regi_num: raise
    soma_refined2regi.append([brain,raw_brain_soma_file,raw_num,brain_soma_file,regi_num]) 
print(len(soma_refined2regi),soma_refined2regi,)

df_brain_somafile = pd.DataFrame(np.array(soma_refined2regi),columns=['brainid','raw_refined_apofile','apo_marker_num','ccf25_regi_swcfile','swc_marker_num'])
df_brain_somafile.to_csv('../assets/brain116_somafile.csv',index=False)

116 [['15257', '/PBshare/SEU-ALLEN/Projects/200k/somalist/22020608/raw_brain/refined/150k/15257_refined.apo', 6308, '/PBshare/SEU-ALLEN/Projects/200k/somalist/22020608/registered/15257_refined_stps.swc', 6308], ['17051', '/PBshare/SEU-ALLEN/Projects/200k/somalist/22020608/raw_brain/refined/50k/17051_total.apo', 305, '/PBshare/SEU-ALLEN/Projects/200k/somalist/22020608/registered/17051_total_stps.swc', 305], ['17052', '/PBshare/SEU-ALLEN/Projects/200k/somalist/22020608/raw_brain/refined/50k/17052_total.apo', 276, '/PBshare/SEU-ALLEN/Projects/200k/somalist/22020608/registered/17052_total_stps.swc', 276], ['17109', '/PBshare/SEU-ALLEN/Projects/200k/somalist/22020608/raw_brain/refined/50k/17109_total.apo', 884, '/PBshare/SEU-ALLEN/Projects/200k/somalist/22020608/registered/17109_total_stps.swc', 884], ['17298', '/PBshare/SEU-ALLEN/Projects/200k/somalist/22020608/raw_brain/refined/150k/17298_refined.apo', 6825, '/PBshare/SEU-ALLEN/Projects/200k/somalist/22020608/registered/17298_refined_stps

In [4]:
raw2regi_somaxyz_df = pd.DataFrame()
somabrain_list = []
raw_somaxyz_list = []
raw_somaregion_list = []
regi_somaxyz_list = []
regi_somaregion_n671_u32_list = []
regi_somaregion_n671_u32_check_list = []
regi_somaregion_list = []
regi_somaregion_check_list = []
for i,brain in enumerate(brains_regi_soma_list):
    if brain in brains_raw_soma_list1:
        brain_soma_file = f'{regi_soma_dir}/{brain}_total_stps.swc'
        raw_brain_soma_file = f'{raw_soma_dir1}/{brain}_total.apo'
    else:
        brain_soma_file = f'{regi_soma_dir}/{brain}_refined_stps.swc'
        raw_brain_soma_file = f'{raw_soma_dir2}/{brain}_refined.apo'
    regi_somaxyz = get_xyz(brain_soma_file)
    raw_somaxyz = get_xyz(raw_brain_soma_file)
    somabrain = [brain]*len(raw_somaxyz)
    raw_somaregion = ['']*len(raw_somaxyz)
    regi_somaregion_n671_u32 = get_n671_u32_region_list(regi_somaxyz)
    regi_somaregion = get_region_name_list(regi_somaregion_n671_u32,32)
    regi_somaregion_n671_u32_check = check_region_u32list(regi_somaregion_n671_u32,n=671)
    regi_somaregion_check = get_region_name_list(regi_somaregion_n671_u32_check,32)

    somabrain_list.extend(somabrain)
    raw_somaxyz_list.extend(raw_somaxyz)
    raw_somaregion_list.extend(raw_somaregion)
    regi_somaxyz_list.extend(regi_somaxyz)
    regi_somaregion_n671_u32_list.extend(regi_somaregion_n671_u32)
    regi_somaregion_n671_u32_check_list.extend(regi_somaregion_n671_u32_check)
    regi_somaregion_list.extend(regi_somaregion)
    regi_somaregion_check_list.extend(regi_somaregion_check)
    print(i,brain,len(regi_somaregion),len(regi_somaregion_list))
raw2regi_somaxyz_df['brainid'] = somabrain_list
raw2regi_somaxyz_df['raw_somaxyz'] = raw_somaxyz_list
raw2regi_somaxyz_df['raw_somaregion'] = raw_somaregion_list
raw2regi_somaxyz_df['regi_somaxyz'] = regi_somaxyz_list
raw2regi_somaxyz_df['regi_somaregion_n671_u32'] = regi_somaregion_n671_u32_list
raw2regi_somaxyz_df['regi_somaregion'] = regi_somaregion_list
raw2regi_somaxyz_df['regi_somaregion_n671_u32_check'] = regi_somaregion_n671_u32_check_list
raw2regi_somaxyz_df['regi_somaregion_check'] = regi_somaregion_check_list

raw2regi_somaxyz_df.to_csv('../assets/soma200k_xyz_region671.csv',index=False)
raw2regi_somaxyz_df

0 15257 6308 6308
1 17051 305 6613
2 17052 276 6889
3 17109 884 7773
4 17298 6825 14598
5 17300 2302 16900
6 17301 1003 17903
7 17302 652 18555
8 17304 201 18756
9 17539 650 19406
10 17541 121 19527
11 17542 3475 23002
12 17543 1552 24554
13 17544 41 24595
14 17545 715 25310
15 17781 606 25916
16 17782 403 26319
17 17783 2083 28402
18 17785 452 28854
19 17786 1696 30550
20 17787 89 30639
21 17788 119 30758
22 18047 115 30873
23 18049 3 30876
24 18052 322 31198
25 18053 790 31988
26 182712 1858 33846
27 182721 1 33847
28 182722 757 34604
29 182726 331 34935
30 18452 40 34975
31 18453 368 35343
32 18454 2626 37969
33 18455 2968 40937
34 18457 311 41248
35 18458 718 41966
36 18459 131 42097
37 18461 75 42172
38 18462 428 42600
39 18463 664 43264
40 18464 248 43512
41 18465 349 43861
42 18466 258 44119
43 18467 508 44627
44 18468 22 44649
45 18469 337 44986
46 18470 220 45206
47 18471 911 46117
48 18472 3099 49216
49 18860 3320 52536
50 18861 2068 54604
51 18862 842 55446
52 18864 520 5596

,brainid,raw_somaxyz,raw_somaregion,regi_somaxyz,regi_somaregion_n671_u32,regi_somaregion,regi_somaregion_n671_u32_check,regi_somaregion_check
0,15257,"[12846.438, 20686.848, 3762.868]",,"[224.133, 136.67, 210.274]",310,SF,310,SF
1,15257,"[12810.299, 20725.111, 3701.844]",,"[221.52, 138.461, 209.876]",603,fi,0,
2,15257,"[12827.447, 20766.654, 3633.077]",,"[218.269, 139.45, 209.441]",603,fi,0,
3,15257,"[9707.336, 28942.707, 4827.952]",,"[279.613, 180.116, 88.527]",672,CP,672,CP
4,15257,"[6884.128, 14090.333, 3235.582]",,"[207.738, 254.492, 308.483]",342,SI,342,SI
...,...,...,...,...,...,...,...,...
227576,236174,"[18176.244, 24154.77, 6192.542]",,"[306.343, 239.819, 332.601]",780,PA,780,PA
227577,236174,"[18117.527, 24033.283, 6208.595]",,"[306.815, 239.014, 330.559]",780,PA,780,PA
227578,236174,"[8812.607, 17915.576, 4825.907]",,"[239.603, 85.751, 219.926]",296,ACAv2/3,296,ACAv2/3
227579,236174,"[9570.51, 7845.859, 6146.163]",,"[297.042, 126.665, 71.783]",156,AUDd6a,156,AUDd6a


In [9]:
raw2regi_somaxyz_df = pd.read_csv('../assets/soma200k_xyz_region671.csv')
plot_df = raw2regi_somaxyz_df[raw2regi_somaxyz_df['regi_somaregion_n671_u32']!=0]
plot_df

,brainid,raw_somaxyz,raw_somaregion,regi_somaxyz,regi_somaregion_n671_u32,regi_somaregion,regi_somaregion_n671_u32_check,regi_somaregion_check
0,15257,"[12846.438, 20686.848, 3762.868]",NaN,"[224.133, 136.67, 210.274]",310,SF,310,SF
1,15257,"[12810.299, 20725.111, 3701.844]",NaN,"[221.52, 138.461, 209.876]",603,fi,0,NaN
2,15257,"[12827.447, 20766.654, 3633.077]",NaN,"[218.269, 139.45, 209.441]",603,fi,0,NaN
3,15257,"[9707.336, 28942.707, 4827.952]",NaN,"[279.613, 180.116, 88.527]",672,CP,672,CP
4,15257,"[6884.128, 14090.333, 3235.582]",NaN,"[207.738, 254.492, 308.483]",342,SI,342,SI
...,...,...,...,...,...,...,...,...
227576,236174,"[18176.244, 24154.77, 6192.542]",NaN,"[306.343, 239.819, 332.601]",780,PA,780,PA
227577,236174,"[18117.527, 24033.283, 6208.595]",NaN,"[306.815, 239.014, 330.559]",780,PA,780,PA
227578,236174,"[8812.607, 17915.576, 4825.907]",NaN,"[239.603, 85.751, 219.926]",296,ACAv2/3,296,ACAv2/3
227579,236174,"[9570.51, 7845.859, 6146.163]",NaN,"[297.042, 126.665, 71.783]",156,AUDd6a,156,AUDd6a


In [9]:
plot_df = raw2regi_somaxyz_df[raw2regi_somaxyz_df['regi_somaregion_n671_u32_check']!=0]
plot_df

,brainid,raw_somaxyz,raw_somaregion,regi_somaxyz,regi_somaregion_n671_u32,regi_somaregion,regi_somaregion_n671_u32_check,regi_somaregion_check
0,15257,"[12846.438, 20686.848, 3762.868]",,"[224.133, 136.67, 210.274]",310,SF,310,SF
3,15257,"[9707.336, 28942.707, 4827.952]",,"[279.613, 180.116, 88.527]",672,CP,672,CP
4,15257,"[6884.128, 14090.333, 3235.582]",,"[207.738, 254.492, 308.483]",342,SI,342,SI
5,15257,"[9347.471, 17057.223, 2969.26]",,"[191.479, 214.211, 260.891]",351,BST,351,BST
8,15257,"[12242.572, 7332.924, 4970.031]",,"[267.488, 125.819, 414.819]",806,SSs2/3,806,SSs2/3
...,...,...,...,...,...,...,...,...
227576,236174,"[18176.244, 24154.77, 6192.542]",,"[306.343, 239.819, 332.601]",780,PA,780,PA
227577,236174,"[18117.527, 24033.283, 6208.595]",,"[306.815, 239.014, 330.559]",780,PA,780,PA
227578,236174,"[8812.607, 17915.576, 4825.907]",,"[239.603, 85.751, 219.926]",296,ACAv2/3,296,ACAv2/3
227579,236174,"[9570.51, 7845.859, 6146.163]",,"[297.042, 126.665, 71.783]",156,AUDd6a,156,AUDd6a


In [10]:
xyzlist = plot_df['regi_somaxyz'].tolist()
obj_dest = 'r255_g0_b0_soma190939.obj' 
write_somaobj_fromxyzlist(obj_dest, xyzlist)

In [11]:
plot_regions_id_list,plot_regions_somanum_list = np.unique(plot_df['regi_somaregion_n671_u32_check'],return_counts=True)
new_index = np.argsort(-plot_regions_somanum_list)
plot_regions_id_list = plot_regions_id_list[new_index].tolist()
zero = len(plot_regions_somanum_list)
first = len(plot_regions_somanum_list[plot_regions_somanum_list>=500])
second = len(plot_regions_somanum_list[plot_regions_somanum_list>=5])
plot_regions_somanum_list = plot_regions_somanum_list[new_index].tolist()
#print(plot_regions_id_list,plot_regions_somanum_list,plot_regions_name_list)
print(zero,first,second)
plot_regions_name_list = get_region_name_list(plot_regions_id_list,uint=32)
plot_regions_rgb_list = get_region_rgb_list(plot_regions_id_list,uint=32)
print(plot_regions_id_list[:first],plot_regions_somanum_list[:first],plot_regions_name_list[:first],plot_regions_rgb_list[:first])

553 64 496
[672, 507, 1091, 1007, 754, 159, 961, 698, 1064, 1025, 56, 936, 828, 1056, 984, 382, 1041, 811, 952, 820, 10703, 951, 342, 313, 957, 477, 1090, 10, 648, 783, 862, 733, 944, 966, 767, 1049, 1033, 976, 943, 778, 583, 800, 1101, 962, 974, 998, 258, 795, 1070, 844, 403, 821, 429, 354, 632, 687, 1102, 463, 638, 128, 1047, 806, 17, 502] [24580, 20468, 7369, 4985, 4938, 4872, 4466, 4261, 4210, 4029, 4021, 3657, 3585, 3303, 2931, 2717, 2327, 2191, 2101, 2030, 1707, 1549, 1518, 1494, 1428, 1263, 1130, 1032, 981, 966, 933, 901, 848, 832, 830, 805, 793, 776, 753, 742, 708, 706, 663, 646, 634, 628, 627, 622, 615, 609, 606, 594, 559, 555, 553, 552, 552, 542, 536, 529, 525, 524, 522, 501] ['CP', 'MOB', 'CUL4, 5', 'SIM', 'OT', 'AON', 'PIR', 'OLF', 'ANcr2', 'PRM', 'ACB', 'DEC', 'ICe', 'ANcr1', 'CENT3', 'CA1', 'PFL', 'ICc', 'EPd', 'ICd', 'DG-mo', 'PYR', 'SI', 'MB', 'UVU', 'STR', 'SSs5', 'SCig', 'MOp5', 'AId6a', 'SSs6a', 'VPM', 'FOTU', 'EPv', 'MOs5', 'FL', 'COPY', 'CENT2', 'MOp2/3', 'VISp5', 

In [12]:
for region in ['MOB','CP','CA1','ICe','SIM']:
    index = plot_regions_name_list.index(region)
    print(region,plot_regions_id_list[index],plot_regions_somanum_list[index])

MOB 507 20468
CP 672 24580
CA1 382 2717
ICe 828 3585
SIM 1007 4985


In [14]:
# plot_regions_id_list = [507,672,382,828,1007]
# plot_regions_rgb_list = get_region_rgb_list(region_list)
for region_name,region_rgb in zip(plot_regions_name_list,plot_regions_rgb_list):
    r,g,b = region_rgb
    obj_dest = 'somaobj/' + f'r{r}_g{g}_b{b}_{region_name}.obj'.replace('/','')
    print(region_name,obj_dest)
    xyzlist = plot_df[plot_df['regi_somaregion_check']==region_name]['regi_somaxyz']
    write_somaobj_fromxyzlist(obj_dest, xyzlist)   

CP somaobj/r152_g214_b249_CP.obj
MOB somaobj/r154_g210_b189_MOB.obj
CUL4, 5 somaobj/r255_g252_b145_CUL4, 5.obj
SIM somaobj/r255_g252_b145_SIM.obj
OT somaobj/r128_g205_b248_OT.obj
AON somaobj/r84_g191_b148_AON.obj
PIR somaobj/r106_g203_b186_PIR.obj
OLF somaobj/r154_g210_b189_OLF.obj
ANcr2 somaobj/r255_g252_b145_ANcr2.obj
PRM somaobj/r255_g252_b145_PRM.obj
ACB somaobj/r128_g205_b248_ACB.obj
DEC somaobj/r255_g252_b145_DEC.obj
ICe somaobj/r255_g122_b255_ICe.obj
ANcr1 somaobj/r255_g252_b145_ANcr1.obj
CENT3 somaobj/r255_g252_b145_CENT3.obj
CA1 somaobj/r126_g208_b75_CA1.obj
PFL somaobj/r255_g252_b145_PFL.obj
ICc somaobj/r255_g122_b255_ICc.obj
EPd somaobj/r160_g238_b157_EPd.obj
ICd somaobj/r255_g122_b255_ICd.obj
DG-mo somaobj/r126_g208_b75_DG-mo.obj
PYR somaobj/r255_g252_b145_PYR.obj
SI somaobj/r162_g177_b216_SI.obj
MB somaobj/r255_g100_b255_MB.obj
UVU somaobj/r255_g252_b145_UVU.obj
STR somaobj/r152_g214_b249_STR.obj
SSs5 somaobj/r24_g128_b100_SSs5.obj
SCig somaobj/r255_g144_b255_SCig.obj
MOp5

VISpm2/3 somaobj/r8_g133_b140_VISpm23.obj
HATA somaobj/r51_g185_b50_HATA.obj
TRS somaobj/r150_g167_b211_TRS.obj
GR somaobj/r255_g165_b210_GR.obj
ENTl1 somaobj/r50_g184_b37_ENTl1.obj
MA somaobj/r162_g177_b216_MA.obj
CS somaobj/r255_g195_b149_CS.obj
VISrl2/3 somaobj/r0_g159_b172_VISrl23.obj
ORBvl5 somaobj/r36_g138_b94_ORBvl5.obj
AHN somaobj/r255_g76_b62_AHN.obj
VISli5 somaobj/r8_g133_b140_VISli5.obj
ACAd6a somaobj/r64_g166_b102_ACAd6a.obj
ARH somaobj/r255_g93_b80_ARH.obj
TMv somaobj/r255_g76_b62_TMv.obj
HPF somaobj/r126_g208_b75_HPF.obj
SSp-un2/3 somaobj/r24_g128_b100_SSp-un23.obj
SOCl somaobj/r255_g174_b111_SOCl.obj
TU somaobj/r242_g72_b59_TU.obj
AUDd1 somaobj/r1_g147_b153_AUDd1.obj
DMH somaobj/r255_g85_b71_DMH.obj
VISal5 somaobj/r8_g133_b140_VISal5.obj
MGv somaobj/r255_g128_b132_MGv.obj
VISl2/3 somaobj/r8_g133_b140_VISl23.obj
MPN somaobj/r255_g76_b62_MPN.obj
SSp-ll1 somaobj/r24_g128_b100_SSp-ll1.obj
VISal2/3 somaobj/r8_g133_b140_VISal23.obj
IO somaobj/r255_g179_b217_IO.obj
VISam5 somao

AUDv6b somaobj/r1_g147_b153_AUDv6b.obj
VISrl6a somaobj/r0_g159_b172_VISrl6a.obj
NLOT3 somaobj/r149_g228_b200_NLOT3.obj
VeCB somaobj/r255_g253_b188_VeCB.obj
STN somaobj/r242_g72_b59_STN.obj
MOs6b somaobj/r31_g157_b90_MOs6b.obj
ILA6a somaobj/r89_g179_b99_ILA6a.obj
PPT somaobj/r255_g144_b255_PPT.obj
ND somaobj/r255_g144_b255_ND.obj
PeF somaobj/r242_g72_b59_PeF.obj
DTN somaobj/r255_g186_b134_DTN.obj
SSp-un6b somaobj/r24_g128_b100_SSp-un6b.obj
FRP6a somaobj/r38_g143_b69_FRP6a.obj
PVpo somaobj/r255_g85_b71_PVpo.obj
IPDM somaobj/r255_g166_b255_IPDM.obj
FF somaobj/r242_g72_b59_FF.obj
RL somaobj/r255_g166_b255_RL.obj
RPO somaobj/r255_g195_b149_RPO.obj
SSp-bfd6b somaobj/r24_g128_b100_SSp-bfd6b.obj
VISpl6a somaobj/r8_g133_b140_VISpl6a.obj
PBG somaobj/r255_g122_b255_PBG.obj
SPFp somaobj/r255_g128_b132_SPFp.obj
OP somaobj/r255_g144_b255_OP.obj
IPR somaobj/r255_g166_b255_IPR.obj
VISpor6b somaobj/r8_g133_b140_VISpor6b.obj
AUDp6b somaobj/r1_g147_b153_AUDp6b.obj
IPA somaobj/r255_g166_b255_IPA.obj
SSp-l

In [18]:
24580/(26018770*0.01*0.01*0.01)
# 501/(2097794*0.01*0.01*0.01)

944.7026127676289

# history

In [ ]:
# plot = {}
# plot_soma = []
# plot_soma_ac = []
# plot_soma_color = []
# somata_num = 0
# plot_num = 0
# filt_num = 0
# total_tree = []
# annotation = sitk.GetArrayFromImage(sitk.ReadImage("../annotation_25.nrrd"))
# with open("../tree.json",'r') as f: mapping = json.loads(f.read())
# for marker_25 in glob.glob("marker_regi_25/*/[1-9]*.swc"):
#     brain = marker_25.split('/')[-2]
#     marker_25_plot = marker_25.replace('marker_regi_25/'+brain,'marker_25_plot')
#     marker_25_filt = marker_25.replace('marker_regi_25/'+brain,'marker_25_filt')
#     tree = parse_swc(marker_25)
#     plot_tree = []
#     filt_tree = []
    
#     for leaf in tree:
#         i,t,x,y,z,r,p = leaf
#         somata_num += 1
        
#         try:
#             ac = annotation[round(z),round(y),round(x),]  
#             if ac==0: raise
#         except:
#             filt_tree.append(leaf)
#             filt_num += 1
#             continue
            
#         plot_tree.append(leaf)
#         plot_num += 1

#         soma = f'200k_{brain}_{round(x)}_{round(y)}_{round(z)}'
#         plot_soma.append(soma)
#         acronym = [j['acronym'] for j in mapping if j['id']==ac][0]
#         plot_soma_ac.append(acronym)
#         color = [j['rgb_triplet'] for j in mapping if j['id']==ac][0]
#         plot_soma_color.append(f'r{color[0]}g{color[1]}b{color[2]}')

#         total_tree.append(leaf)
#         if acronym in plot.keys(): plot[acronym].append(leaf)
#         else: plot[acronym] = [leaf]
            
#     write_swc(plot_tree,marker_25_plot)
#     write_swc(filt_tree,marker_25_filt)
#     print(marker_25,brain,len(tree),len(plot_tree),len(filt_tree))
# write_swc(plot_total,'total.swc')
# print(somata_num,plot_num,filt_num)

# write_somaobj_fromtree('./marker_obj/total.obj',total_tree)

In [ ]:
# #NOTE
# values = [[round(int(c)/40) for c in s.split('_')[-3:]] for s in plot_soma]
# soma_values = [str(i[0]).zfill(3)+str(i[1]).zfill(3)+str(i[2]).zfill(3) for i in values]
# soma_value,soma_count = np.unique(soma_values,return_counts=True)
# print(np.sort(soma_count))
# count_list = list(range(0,max(soma_count),300))
# count_list.reverse()
# for i in count_list:
#     print(i)

In [ ]:
# df_200ksoma = pd.DataFrame(np.array([plot_soma,plot_soma_ac,plot_soma_color]).T,columns=['soma','acronym','color'])
# df_200ksoma.to_csv('./soma_ac_rgb.csv')
# df_200ksoma

In [ ]:
# acs_200ksoma,acscount_200ksoma = np.unique(df_200ksoma['acronym'],return_counts=True)
# acscount_200ksoma = [(acs_200ksoma[i],acscount_200ksoma[i],df_200ksoma.loc[df_200ksoma['acronym']==acs_200ksoma[i],'color'].values[0]) for i in range(len(acs_200ksoma))]
# acscount_200ksoma.sort(key=lambda x:-x[1])
# print(acscount_200ksoma)

In [ ]:
# colors_200ksoma = np.unique(df_200ksoma['color'])
# classic_color_ac = {}
# for color in colors_200ksoma:
#     classic_color_ac[color] = [i[0] for i in acscount_200ksoma if i[2]==color][0]
# classic_color_ac
# #paraview for classic region plot

In [ ]:
# fig,axes = plt.subplots()
# plot_acs = ['','MOB','SI','CP','CA1','ICe','SIM','arb']
# array=np.ones((1,8,3))
# for ii,ac in enumerate(plot_acs):
#     if ii==0: array[0,ii]=[255,255,255]; continue
#     count,rgb = [i[1:] for i in acscount_200ksoma if ac==i[0]][0]
#     print(rgb)
#     r = int(rgb.split('g')[0].split('r')[-1])
#     g = int(rgb.split('b')[0].split('g')[-1])
#     b = int(rgb.split('b')[-1])
#     array[0,ii]=[r,g,b]
# array = array.astype(np.uint8)
# plt.imshow(array)
# axes.spines['top'].set_visible(False)
# axes.spines['left'].set_visible(False)
# axes.spines['right'].set_visible(False)
# plt.xticks([0,1,2,3,4,5,6,7],plot_acs,fontsize=24)
# plt.xlabel('soma region                  ',fontsize=26,fontweight='bold')
# plt.yticks(ticks=[])
# plt.savefig(f'classic.png')

In [ ]:
# for ii,ac in enumerate(plot_acs):
#     count,rgb = [i[1:] for i in acscount_200ksoma if ac==i[0]][0]
#     vtk = '../vtk/' + ac + '.vtk'
#     if not os.path.exists(vtk): continue
        
#     plot_vtk = './marker_obj/' + ac + '_' + rgb + '.vtk'
#     cp_vtk_str = f'cp {vtk} {plot_vtk}'
#     os.system(cp_vtk_str)
    
#     plot_obj = './marker_obj/' + ac + '_' + rgb + '.obj'
#     write_somaobj_fromtree(plot_obj,plot[ac])
#     print(plot_obj)
# #mannual paraview 7 ac plot   